In [7]:
import numpy as np
import sys

from scipy.linalg import eig
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigs
from numpy import allclose

path = 'D:/repositories/composipy'

sys.path.append(path)

from time import time

from composipy import OrthotropicMaterial, LaminateProperty
from composipy import PlateStructure



In [33]:
E1 = 60800
E2 = 58250
v12 = 0.07
G12 = 4550
t = 0.21

stacking = [45,-45,0,90,45,-45,0,90,45,-45]
stacking += stacking[::-1]

a = 360.
b = 360.

In [69]:
ply_1 = OrthotropicMaterial(E1, E2, v12, G12, t)

l1 = LaminateProperty(stacking, ply_1)


ply1 = OrthotropicMaterial(129500, 9370, 0.38, 5240, 0.2)
stacking = [-45, 45, 90, 0, 0, 0, 0, 90, 45, -45]
laminate1 = LaminateProperty(stacking, ply1)



constraints={
                'x0': ['TX', 'TY', 'TZ', 'RX', 'RY', 'RZ'],
                'xa': ['TX', 'TY', 'TZ', 'RX', 'RY', 'RZ'],
                'y0': ['TX', 'TY', 'TZ', 'RX', 'RY', 'RZ'],
                'yb': ['TX', 'TY', 'TZ', 'RX', 'RY', 'RZ'],
            }

panel = PlateStructure(laminate1, a, b, m=12, n=12, Nxx=-1, constraints=constraints)

ti = time()
print('calculating K and KG')
K_KG = panel.calc_K_KG()
print(f'calculation done in {time()-ti} seconds')
print('calculating panel eig value')
ti = time()
print(panel.buckling_analysis())
print(f'calculation done in {time()-ti} seconds')

calculating K and KG
calculation done in 4.642505884170532 seconds
calculating panel eig value
27.789595510548764
calculation done in 4.798723220825195 seconds


## comparison with compmech

In [62]:
from compmech.panel.panel import Panel
from compmech.analysis import lb, static

In [66]:
# skin panels
laminaprop = (E1, E2, v12, G12, G12, G12)
p1 = Panel(group='skin', Nxx=-1, x0=0, y0=0., a=a, b=b,m=12, n=12, plyt=t, stack=stacking, laminaprop=laminaprop)

In [67]:
p1.u1tx = 0 ; p1.u1rx = 0 ; p1.u2tx = 0 ; p1.u2rx = 0
p1.v1tx = 0 ; p1.v1rx = 0 ; p1.v2tx = 0 ; p1.v2rx = 0
p1.w1tx = 0 ; p1.w1rx = 0 ; p1.w2tx = 0 ; p1.w2rx = 0
p1.u1ty = 0 ; p1.u1ry = 0 ; p1.u2ty = 0 ; p1.u2ry = 0
p1.v1ty = 0 ; p1.v1ry = 0 ; p1.v2ty = 0 ; p1.v2ry = 0
p1.w1ty = 0 ; p1.w1ry = 0 ; p1.w2ty = 0 ; p1.w2ry = 0


In [68]:
k0_1 = p1.calc_k0()
kG0_1 = p1.calc_kG0()

eigvals, eigvecs = lb(k0_1, kG0_1, silent=True)
eigvals

		Calculating k0... 
		finished!
		Calculating kG0... 
		finished!


array([ 28.26517318,  31.85485655,  50.93565892,  68.80253977,
        74.62550061,  78.03534029, 101.56916464, 103.07497659,
       114.03777715, 137.33253658, 146.78985857, 149.58515026,
       153.59630782, 166.8025119 , 179.09936315, 180.54280735,
       226.43722417, 244.7073578 , 248.42623777, 282.1817778 ,
       283.37977543, 328.11167084, 345.11693194, 365.8228319 ,
       376.31320955])